<h1>Praktikum IF4071</h1>
<p>13515032 Helena Suzane Graciella</p>
<p>13515034 Roselina Pradjanata</p>

In [1]:
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.model_selection import KFold
import pandas as pd
import numpy as np

In [2]:
raw_df = pd.read_csv('CensusIncome/CencusIncome.data.txt', sep=',\s', header=None, na_values=["?"], engine='python')
raw_df.dropna(inplace=True)

raw_test_df = pd.read_csv('CensusIncome/CencusIncome.test.txt', sep=',\s', header=None, na_values=["?"], engine='python', skiprows=1)
raw_test_df.dropna(inplace=True)

In [3]:
income_df = pd.DataFrame()
income_test_df = pd.DataFrame()
le = {}
for feature in raw_df:
    le[feature] = LabelEncoder()
    income_df[feature] = le[feature].fit_transform(raw_df[feature])
    income_test_df[feature] = le[feature].fit_transform(raw_test_df[feature])
income_X = income_df.iloc[:, 0:14]
income_Y = income_df.iloc[:, 14]
income_test_X = income_test_df.iloc[:, 0:14]
income_test_Y = income_test_df.iloc[:, 14]

<p>Setelah dianalisis, dataset mempunyai kolom bertipe string dan bertipe integer. Fungsi clustering pada sklearn tidak dapat memproses data bertipe string, sehingga data bertipe string harus diubah ke integer. Dataset tersebut juga mempunyai instance yang kolomnya memiliki missing value. Nilai tersebut dapat ditangani dengan beberapa cara, antara lain dengan menghilangkan instance yang memiliki missing value, atau dengan mengubah missing value tersebut dengan nilai lain.
Penanganan data yang dilakukan pertama-tama adalah dengan mengubah label data bertipe string ke bertipe integer dengan label encoder. Selanjutnya, dilakukan penanganan instance yang kolomnya mengandung missing value (?) dengan menghilangkan instance tersebut.</p>

In [4]:
income_kf = KFold(n_splits=10)

In [89]:
def purity(clustered, y_true):
    class_0 = []
    class_1 = []
    
    for i in range(0, len(y_true)):
        if (clustered[i] == 0):
            class_0.append(y_true[i])
        else:
            class_1.append(y_true[i])
    
    class_0_mode = max(set(class_0), key=class_0.count)
    class_1_mode = max(set(class_1), key=class_1.count)
    
    n_mode_0 = 0
    for instance in class_0:
        if (instance == class_0_mode):
            n_mode_0 += 1
            
    n_mode_1 = 0
    for instance in class_1:
        if (instance == class_1_mode):
            n_mode_1 += 1
            
    purity_total = (n_mode_0 + n_mode_1)/len(clustered)
    return purity_total

<h2>K-Means</h2>

In [97]:
algos = ['auto', 'full', 'elkan']

best_model = {}
max_purity = 0
for algo in algos:
    for train_index, test_index in income_kf.split(income_X):
        income_k_means = KMeans(n_clusters=2, algorithm=algo)
        income_X_train, income_X_test = income_X.values[train_index], income_X.values[test_index]
        income_Y_train, income_Y_test = income_Y.values[train_index], income_Y.values[test_index]
        income_k_means.fit(income_X_train, income_Y_train)
        purity_score = purity(income_k_means.predict(income_X_train), income_Y_train)
        if (purity_score >= max_purity):
            best_model = income_k_means
            max_purity = purity_score
    print(best_model)
    print("Purity = " + str(max_purity))
print(best_model)
print("Purity = " + str(max_purity))

income_full_X = income_X.append(income_test_X)
income_full_Y = income_Y.append(income_test_Y)
best_model = {}
max_purity = 0
for algo in algos:
    for train_index, test_index in income_kf.split(income_full_X):
        income_k_means = KMeans(n_clusters=2, algorithm=algo)
        income_X_train, income_X_test = income_full_X.values[train_index], income_full_X.values[test_index]
        income_Y_train, income_Y_test = income_full_Y.values[train_index], income_full_Y.values[test_index]
        income_k_means.fit(income_X_train, income_Y_train)
        purity_score = purity(income_k_means.predict(income_X_train), income_Y_train)
        if (purity_score >= max_purity):
            best_model = income_k_means
            max_purity = purity_score
    print(best_model)
    print("Purity = " + str(max_purity))
print(best_model)
print("Purity = " + str(max_purity))

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=2, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)
Purity = 0.7521827224166513
KMeans(algorithm='full', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=2, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)
Purity = 0.7521827224166513
KMeans(algorithm='elkan', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=2, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)
Purity = 0.7521827224166513
KMeans(algorithm='elkan', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=2, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)
Purity = 0.7521827224166513
KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=2, n_init=10, n_jobs=1, precompute_distances='auto',
    random_s

<p>Dari eksperimen di atas, didapatkan bahwa algoritma terbaik untuk model K-Means adalah:</p>

<p><i>KMeans(algorithm='elkan', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=2, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)</i></p>
    
<p>untuk kedua dataset. Namun demikian, setelah melakukan eksperimen, didapatkan bahwa purity setiap algoritma ('auto'/'full'/'elkan') adalah sama.</p>

<p>Pada eksperimen ini, dilakukan 10-fold cross validation untuk mendapatkan nilai purity terbaik.</p>

In [103]:
linkages = ['ward', 'complete', 'average']
income_part_X = income_X.iloc[0:10000, :]
income_part_Y = income_Y.iloc[0:10000]
best_model = {}
max_purity = 0
for link in linkages:
    income_agglomerative = AgglomerativeClustering(n_clusters=2, linkage=link)
    income_agglomerative.fit(income_part_X)
    purity_score = purity(income_agglomerative.fit_predict(income_part_X), income_part_Y)
    if (purity_score >= max_purity):
        best_model = income_agglomerative
        max_purity = purity_score
    print(best_model)
    print("Purity = " + str(max_purity))
print(best_model)
print("Purity = " + str(max_purity))

AgglomerativeClustering(affinity='euclidean', compute_full_tree='auto',
            connectivity=None, linkage='ward', memory=None, n_clusters=2,
            pooling_func=<function mean at 0x7f7a7c082488>)
Purity = 0.755
AgglomerativeClustering(affinity='euclidean', compute_full_tree='auto',
            connectivity=None, linkage='complete', memory=None,
            n_clusters=2, pooling_func=<function mean at 0x7f7a7c082488>)
Purity = 0.755
AgglomerativeClustering(affinity='euclidean', compute_full_tree='auto',
            connectivity=None, linkage='average', memory=None,
            n_clusters=2, pooling_func=<function mean at 0x7f7a7c082488>)
Purity = 0.755
AgglomerativeClustering(affinity='euclidean', compute_full_tree='auto',
            connectivity=None, linkage='average', memory=None,
            n_clusters=2, pooling_func=<function mean at 0x7f7a7c082488>)
Purity = 0.755


<p>Dari eksperimen di atas, didapatkan bahwa linkage terbaik untuk model Agglomerative Clustering adalah:</p>

<p><i>AgglomerativeClustering(affinity='euclidean', compute_full_tree='auto',
            connectivity=None, linkage='ward', memory=None, n_clusters=2)</i></p>
    
<p>Namun demikian, setelah melakukan eksperimen, didapatkan bahwa purity setiap linkage ('ward'/'complete'/'average') adalah sama.</p>

<p>Pada eksperimen ini, data dipotong menjadi 10.000 instances karena mesin yang digunakan tidak mampu menganalisis lebih dari 10000 instances. Mesin juga tidak dapat melakukan k-fold pada sekitar 30.000 instances. Karena itu, mesin juga tidak mampu melakukan analisis pada data training + data testing.</p>

<h2>DBSCAN</h2>

In [91]:
income_part_X =  income_X.iloc[0:10000, :]
income_part_Y = income_Y.iloc[0:10000]
best_model = {}
max_purity = 0

best_eps = {}
for eps in range(40, 50):
    dbscan_model = DBSCAN(eps=45, min_samples=4).fit(income_part_X)
    purity_score = purity(dbscan_model.fit_predict(income_part_X), income_part_Y)
    if (purity_score > max_purity):
        best_model = dbscan_model
        max_purity = purity_score
        best_eps = eps
        
print(best_model)
print("Purity = " + str(max_purity))

DBSCAN(algorithm='auto', eps=45, leaf_size=30, metric='euclidean',
    metric_params=None, min_samples=4, n_jobs=1, p=None)
Purity = 0.7935


<p>Dari eksperimen di atas, didapatkan bahwa epsilon terbaik untuk model DBSCAN adalah:</p>
<p><i>DBSCAN(algorithm='auto', eps=45, leaf_size=30, metric='euclidean', metric_params=None, min_samples=4, n_jobs=1, p=None)</i></p>
<p>Pada eksperimen ini, data dipotong menjadi 10.000 instances karena mesin yang digunakan tidak mampu menganalisis lebih dari 10000 instances. Mesin juga tidak dapat melakukan k-fold pada sekitar 30.000 instances. Karena itu, mesin juga tidak mampu melakukan analisis pada data training + data testing.</p>